In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat


In [2]:
plate = 13
date_init = datetime(2020,7,1,19,57)
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(4)]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [3]:
class Experiment():
    def __init__(self,plate):
        self.plate = plate
        self.path_param = '//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat'
    def load(self,dates,local=False,pickle=False,raw=False):
        self.dates=dates
        self.raw=raw
        if local:
            paths=[f'Data/graph_{date}_{plate}_full_labeled.csv' for date in dates]
            paths_connection = [f'Data/connection_{date}_{plate}.csv' for date in dates[:-1]]
        else:
            paths = [get_path(date,self.plate,True,extension='_full_labeled.csv') for date in dates]
            paths_connection = [get_path(date,plate,True,extension='_connection.csv') for date in dates[:-1]]
        if raw:
            nx_graph_poss=[]
            for date in dates:
                nx_graph_poss.append(generate_nx_graph(pd.read_csv(get_path(date,plate,True,extension='_raw_aligned_skeleton.csv'),
                                                converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}),labeled=True))
        else:
            nx_graph_poss=[]
            for path in paths:
                nx_graph_poss.append(generate_nx_graph(pd.read_csv(path,
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,
                                                'pixel_list' : ast.literal_eval}),labeled=True))
        from_tip_growth_pattern=[]
        for path in paths_connection:
            from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(path)))
        nx_graphs=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
        poss = [nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
        nx_graph_clean=[]
        for graph in nx_graphs:
            S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
            len_connected=[len(nx_graph.nodes) for nx_graph in S]
            nx_graph_clean.append(S[np.argmax(len_connected)])
        skeletons=[]
        for nx_graph in nx_graph_clean:
            skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))
        connections = [c[0] for c in from_tip_growth_pattern]
        growth_patterns = [c[1] for c in from_tip_growth_pattern]
        self.positions=poss
        self.nx_graph=nx_graph_clean
        self.connections = connections
        self.growth_patterns = growth_patterns
        self.skeletons=skeletons
        labels = {node for g in self.nx_graph for node in g}
        self.nodes=[]
        for label in labels:
            self.nodes.append(Node(label,self))
        
    def copy(self,experiment):
        self.positions=experiment.positions
        self.nx_graph=experiment.nx_graph
        self.connections = experiment.connections
        self.growth_patterns = experiment.growth_patterns
        self.skeletons=experiment.skeletons
        self.dates=experiment.dates
        self.plate=experiment.plate
    def save(self,path=f'Data/'):
        from_tip_growth_pattern_tab=[]
        for date in self.dates[:-1]:
            from_tip_growth_pattern_tab.append(pd.read_csv(get_path(date,plate,True,extension='_connection.csv')))
        for i, date in enumerate(self.dates[:-1]):
            from_tip_growth_pattern_tab[i].to_csv(path+f'connection_{date}_{plate}.csv')
        tabs_labeled=[]
        for date in self.dates:
            tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                        converters={'origin_pos' : transform_list,
                                                    'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))
        for i, date in enumerate(self.dates):
            tabs_labeled[i].to_csv(path+f'graph_{date}_{plate}_full_labeled.csv')
            sio.savemat(path+f'graph_{date}_{plate}_full_labeled.mat', {name: col.values for name, col in tabs_labeled[i].items()})
    def pickle_save(self):
        pickle.dump(self,open( f'Data/'+f"experiment_{plate}.pick", "wb" ))
    def pickle_load(self):
        self = pickle.load( open( f'Data/'+f"experiment_{plate}.pick", "rb" ) )
    def get_node(self,label):
        return(Node(label,self))
    def get_edge(self,begin,end):
        return(Edge(begin,end,self))
    def compress_skeleton(self,t,factor):
        shape=self.skeletons[t].shape
        final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
        for pixel in self.skeletons[t].keys():
            x=min(round(pixel[0]/factor),shape[0]//factor-1)
            y=min(round(pixel[1]/factor),shape[1]//factor-1)
            final_picture[x,y]+=1
        return(final_picture>=1)
    def load_images(self,t):
        date = self.dates[t]
        date_plate = f'/2020{date}'
        plate_str = f'_Plate{self.plate}'
        path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
        for name in tileconfig[0]:
            im=imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name)
            np.save('Temp/'+name,im)
    def find_image_pos(self,xs,ys,t,local=False):
        params=read_mat(self.path_param)
        x1=params['x1']
        x2=params['x2']
        y0=params['y0']
        Sf=params['Sf']
        L0center=params['L0center']
        date = self.dates[t]
        date_plate = f'/2020{date}'
        plate_str = f'_Plate{self.plate}'
        findingbaits = read_mat('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Analysis/FindingBaits.mat')
        Rcenter=findingbaits['Rcenter']
        Rradii = findingbaits['Rradii']
        Lcenter = findingbaits['Lcenter']
        t = findingbaits['t']
        Lradii = findingbaits['Lradii']
        path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
        xs_yss=list(tileconfig[2])
        xes= [xs_ys[0] for xs_ys in xs_yss]
        yes= [xs_ys[1] for xs_ys in xs_yss]
        cmin=np.min(xes)
        cmax=np.max(xes)
        rmin = np.min(yes)
        rmax = np.max(yes)
        S0     = [rmax+3000+abs(rmin),cmax+4096+abs(cmin)]
        theta  = np.arctan((Rcenter[1]-Lcenter[1])/(Rcenter[0]-Lcenter[0]))
        Sry    =  S0[0]*np.cos(abs(theta))+S0[1]*np.sin(abs(theta))
        C = np.tan(-np.arctan((ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                               *np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/
                              (xs + (x2+x1)/2 - (Sf[0]-y0) - 1 - L0center[0] + np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                               *np.cos(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))) - theta)
        ximg = - (ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                  *np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/(np.sqrt(1+C**2)*np.sin(np.arctan(C)+theta)) 
        yimg = S0[0] - C*ximg
        def find(xsub,ysub,x,y):
            indexes=[]
            for i in range(len(xsub)):
                if x>=xsub[i]-cmin and x<xsub[i]-cmin+4096 and y>=ysub[i]-rmin and y < ysub[i]-rmin+3000:
                    indexes.append(i)
            return(indexes)
        indsImg = find(xes,yes,ximg,yimg)
        possImg  = [ximg-np.array(xes)[indsImg]+cmin+1, yimg-np.array(yes)[indsImg]+rmin+1]
        paths=[]
        for index in indsImg:
            if local:
                paths.append('Temp/'+tileconfig[0][index]+'.npy')
            else:
                paths.append('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+tileconfig[0][index])
        if local:
             ims = [np.load(path) for path in paths]
        else:
            ims = [imageio.imread(path) for path in paths]
        return(ims,possImg)
    def plot(self,ts,labels=[],relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=5,save='',time=None):
        global check
        right = 0.90
        top = 0.90
        if len(save)>=1:
            fig=plt.figure(figsize=(14,12))
            size = 10
        else:
            fig = plt.figure()
            size = 5
        ax = fig.add_subplot(111)
        scale=1/len(ts)
        power=len(ts)
        compressed_images=[]
        for i,t in enumerate(ts):
            compressed_images.append(self.compress_skeleton(t,compress))
        visibility = [True for t in ts]
        final_image=scale*compressed_images[0]*visibility[0]
        for i,compressed_image in enumerate(compressed_images[1:]):
            final_image=np.power(final_image**power+((i+2)*scale*compressed_image*visibility[i+1])**power,1/power)
        l1=ax.imshow(final_image, cmap='plasma',interpolation='none',vmin=0,vmax=1)
        rax = plt.axes([0.05, 0.4, 0.1, 0.15])
        labels=[f'{4*t}h' for t in ts]
        check = CheckButtons(rax, labels, visibility)
        def func(label):
            index = labels.index(label)
            visibility[index]= not visibility[index]
            final_image=visibility[0]*scale*compressed_images[0]
            for i,compressed_image in enumerate(compressed_images[1:]):
                final_image=np.power(final_image**power+visibility[i+1]*((i+2)*scale*compressed_image)**power,1/power)
            l1.set_data(final_image)
            plt.draw()

        check.on_clicked(func)

#         bbox_props1 = dict(boxstyle="circle", fc="grey")
#         bbox_props2 = dict(boxstyle="circle", fc="white")
#         ax.text(right, top, time,
#             horizontalalignment='right',
#             verticalalignment='bottom',
#             transform=ax.transAxes,color='white')
#         for node in node_list_t:
#             t = ax.text((pos_t[node][1]-shift[1])//compress, (pos_t[node][0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
#                         size=size,
#                         bbox=bbox_props1)
#         for node in node_list_tp1:
#             if node in pos_tp1.keys():
#                 t = ax.text((pos_tp1[node][1]-shift[1])//compress, (pos_tp1[node][0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
#                             size=size,
#                             bbox=bbox_props2)
        if len(save)>=1:
            plt.savefig(save)
            plt.close(fig)
        else:
            plt.show()
        

class Node():
    def __init__(self,label,experiment):
        self.experiment = experiment
        self.label = label
    def __repr__(self):
        return (f'Node({self.label})')
    def __str__(self):
        return str(self.label)
    def neighbours(self,t):
        return([self.experiment.get_node(node) for node in self.experiment.nx_graph[t].neighbors(self.label)])
    def is_in(self,t):
        return(self.label in self.experiment.nx_graph[t].nodes)
    def degree(self,t):
        return(self.experiment.nx_graph[t].degree(self.label))
    def edges(self,t):
        return([self.experiment.get_edge(self,neighbour) for neighbour in self.neighbours(t)])
    def pos(self,t):
        return(self.experiment.positions[t][self.label])

class Edge():
    def __init__(self,begin,end,experiment):
        self.begin=begin
        self.end = end
        self.experiment = experiment
    def __repr__(self):
        return (f'Edge({self.begin},{self.end})')
    def __str__(self):
        return str((self.begin,self.end))
    def is_in(self,t):
        return((self.begin,self.end) in self.experiment.nx_graph[t].edges)
    def pixel_list(self,t):
        return(orient(self.experiment.nx_graph[t].get_edge_data(self.begin.label,self.end.label)['pixel_list'],self.begin.pos(t)))
    def orientation_whole(self,t):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[-1]-pixel_list[0]
        unit_vector = vector/np.linalg.norm(vector)
        vertical_vector=np.array([-1,0])
        dot_product = np.dot(vertical_vector,unit_vector)
        if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
            angle = np.arccos(dot_product)/(2*np.pi)*360
        else:
            angle = -np.arccos(dot_product)/(2*np.pi)*360
        return(angle)
        

In [5]:
exp = Experiment(13)

In [6]:
exp.load(dates,local=False,raw=True)

In [21]:
len(experiment.nx_graph[0].nodes)

2953

In [8]:
experiment_updated= Experiment(13)
experiment_updated.load(dates,local=False,raw=False)

In [17]:
experiment_updated.save()

wow


In [88]:
# experiment_updated= Experiment(13)
# experiment_updated.copy(experiment)

In [29]:
ims,poss=experiment_updated.find_image_pos(5384,15468,0,local=True)

In [30]:
len(ims)

2

In [31]:
i=1
plot_t_tp1([0],[],{0:(poss[1][i],poss[0][i])},None,ims[i],ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
[14350 15103]

In [97]:
15468/5,5384/5

(3093.6, 1076.8)

In [26]:
plt.close('all')

In [12]:
exp.plot([0,1,2],compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0040756688963210705
0.004225752508361204
0.004315713837792642


In [13]:
experiment_updated.plot([0,1,2],compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.00390928093645485
0.003942386078595318
0.004090327132107024


In [19]:
plt.close('all')

In [ ]:
[17131  2149]
[17131  2172]
[17132  2125]
[14406 15168]
[17133 28106]
[17138 25352]
[16152 13771]
[17142  6483]
[16160 16226]
[14426 15145]
[14446 15167]
[19509 42399]
[17151  2169]
[17154 28150]
[17155  2134]
[14417  6078]
[17138 25352]

In [36]:
25352/5,17138 /5

(5070.4, 3427.6)

In [37]:
experiment.plot([2],compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
experiment_updated.load_images(0)

In [247]:
experiment_updated

In [264]:
experiment_updated.pickle_save()

In [19]:
experiment_updated.pickle_load()

In [124]:
experiment.get_node(19).edges(0)[1].orientation_whole(0)

62.300527191945015

In [120]:
experiment.get_node(19).edges(0)

[Edge(19,25), Edge(19,16), Edge(19,20)]

In [44]:
list(experiment.nodes[].neighbours(0))

[2]

In [16]:
experiment.positions[0][0]

array([9248, 6724])

In [72]:
experiment.save()

wow


In [139]:
np.random.seed(1)
random_color()

[37, 235, 140]

In [133]:
import random
def random_color():
    return list(np.random.choice(range(256), size=3))

In [237]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [15]:
def plotiplot():
    t = np.arange(0.0, 2.0, 0.01)
    s0 = np.sin(2*np.pi*t)
    s1 = np.sin(4*np.pi*t)
    s2 = np.sin(6*np.pi*t)

    fig, ax = plt.subplots()
    l0, = ax.plot(t, s0, visible=False, lw=2, color='k', label='2 Hz')
    l1, = ax.plot(t, s1, lw=2, color='r', label='4 Hz')
    l2, = ax.plot(t, s2, lw=2, color='g', label='6 Hz')
    plt.subplots_adjust(left=0.2)

    lines = [l0, l1, l2]

    # Make checkbuttons with all plotted lines with correct visibility
    rax = plt.axes([0.05, 0.4, 0.1, 0.15])
    labels = [str(line.get_label()) for line in lines]
    visibility = [line.get_visible() for line in lines]
    check = CheckButtons(rax, labels, visibility)


    def func(label):
        index = labels.index(label)
        lines[index].set_visible(not lines[index].get_visible())
        plt.draw()

    check.on_clicked(func)

#     plt.show()

In [16]:
plotiplot()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
a1=np.array([[1,0],[0,0]])
a2=np.array([[0,1],[0,0]])
visibility = [True for i in range(2)]
def plotitplo():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    compressed=[a1,a2]
    l1=ax.imshow(a1+a2)
    rax = plt.axes([0.05, 0.4, 0.1, 0.15])
    labels=[f'{4*i}h' for i in range(2)]
    check = CheckButtons(rax, labels, visibility)
    def func(label):
        index = labels.index(label)
        check.set_active(index)
        visib=check.get_status()
        image_final=compressed[0]*visib[0]
        for i, a in enumerate(compressed[1:]):
            image_final+=a*visib[i+1]
        l1.set_data(image_final)
        plt.draw()

    check.on_clicked(func)

#     plt.show()

In [12]:
plotitplo()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
a1=np.array([[1,0],[0,0]])
a2=np.array([[0,1],[0,0]])
visibility = [True for i in range(2)]
fig = plt.figure()
ax = fig.add_subplot(111)
compressed=[a1,a2]
l1=ax.imshow(a1+a2)
rax = plt.axes([0.05, 0.4, 0.1, 0.15])
labels=[f'{4*i}h' for i in range(2)]
check = CheckButtons(rax, labels, [True for i in range(2)])
# check.set_active(0)
def func(label):
    index = labels.index(label)
    check.set_active(index)
    visib=check.get_status()
    image_final=compressed[0]*visib[0]
    for i, a in enumerate(compressed[1:]):
        image_final+=a*visib[i+1]
    l1.set_data(image_final)
    plt.draw()
    
check.on_clicked(func)

plt.show()

NameError: name 'np' is not defined

In [24]:
visibility

[True, True]

In [2]:
Variable()

NameError: name 'Variable' is not defined